In [1]:
#Importaciones necesarias
import pandas as pd
import gzip
import json

In [2]:
# Ruta al dataset
path = '/content/drive/MyDrive/steam_games.json.gz'

# Descomprimimos el archivo con gzip y leemos linea por linea el JSON.
data = []
with gzip.open(path, 'r') as f:
    for line in f:
      data.append(json.loads(line))

# Convertimos a DataFrame
df_games = pd.DataFrame(data)
df_games.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# Damos una vista rapida a los tipos de datos y verificamos nulos.
df_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher     24083 non-null  object
 1   genres        28852 non-null  object
 2   app_name      32133 non-null  object
 3   title         30085 non-null  object
 4   url           32135 non-null  object
 5   release_date  30068 non-null  object
 6   tags          31972 non-null  object
 7   reviews_url   32133 non-null  object
 8   specs         31465 non-null  object
 9   price         30758 non-null  object
 10  early_access  32135 non-null  object
 11  id            32133 non-null  object
 12  developer     28836 non-null  object
dtypes: object(13)
memory usage: 11.9+ MB


In [4]:
# Observamos que hay bastantes nulos.
# Vamos a eliminarlos tomando como referencia la columna 'id' que contiene el identificador unico de cada item.
df_games.dropna(subset='id', inplace=True)
df_games.reset_index(drop=True, inplace=True)
df_games.shape

(32133, 13)

In [5]:
#Comprobamos si 'id' tiene duplicados.
df_games['id'].duplicated().sum()

1

In [6]:
#Notamos que solo hay un 'id' duplicado y procedemos a elimnarlo.
df_games.drop_duplicates(subset='id', inplace=True)
df_games.shape

(32132, 13)

In [7]:
# Intentamos convertir el tipo de datos de 'id' a 'int'.
df_games['id'] = df_games['id'].astype(int)

In [10]:
# Commprobamos el tipo de dato.
print(df_games['id'].dtypes)

int64


In [11]:
# Cambiamos el nombre de la columna 'id' por 'item_id' que es mas representativo.
df_games.rename(columns={'id': 'item_id'}, inplace=True)
df_games.columns

Index(['publisher', 'genres', 'app_name', 'title', 'url', 'release_date',
       'tags', 'reviews_url', 'specs', 'price', 'early_access', 'item_id',
       'developer'],
      dtype='object')

In [13]:
# Eliminamos las columnas 'url' y 'reviews_url' que son irrelevantes para nuestro proyecto.
df_games.drop(['url', 'reviews_url'], axis=1, inplace=True)
df_games.columns

Index(['publisher', 'genres', 'app_name', 'title', 'release_date', 'tags',
       'specs', 'price', 'early_access', 'item_id', 'developer'],
      dtype='object')

In [14]:
#Exploramos el contenido de 'title' y 'app_name'
df_games[['title', 'app_name']]

,title,app_name
0,Lost Summoner Kitty,Lost Summoner Kitty
1,Ironbound,Ironbound
2,Real Pool 3D - Poolians,Real Pool 3D - Poolians
3,弹炸人2222,弹炸人2222
4,NaN,Log Challenge
...,...,...
32128,Colony On Mars,Colony On Mars
32129,LOGistICAL: South Africa,LOGistICAL: South Africa
32130,Russian Roads,Russian Roads
32131,EXIT 2 - Directions,EXIT 2 - Directions


In [15]:
# A simple vista parece que 'app_name' y 'title' son iguales, vamos a comprobarlo.
# Realizamos la comparación y creamos una columna con el resultado.
df_games['is_equal'] = df_games['app_name'] == df_games['title']

# Calculamos el porcentaje de filas donde 'app_name' y 'title' son iguales
equal_percentage = round((df_games['is_equal'].sum() / len(df_games)) * 100, 2)

print(f"El porcentaje de filas donde 'app_name' y 'title' son iguales es: {equal_percentage}%")

El porcentaje de filas donde 'app_name' y 'title' son iguales es: 91.9%


In [16]:
# Al comprobar que son iguales en mas de un 91% eliminamos 'title' que tiene mayor cantidad de nulos.
df_games = df_games.drop(['title', 'is_equal'], axis=1)

In [17]:
# En el analisis previo obrservamos que 'app_name' tenia algunos nulos que remplazaremos por 'unknown'.
df_games['app_name'].fillna('unknown', inplace=True)

# Intentamos convertir a tipo 'string'.
df_games['app_name'] = df_games['app_name'].astype('string')

# Tambien cambiaremos el nombre de la columna por uno mas representativo como 'item_name'
df_games.rename(columns={'app_name':'item_name'}, inplace=True)

In [18]:
df_games.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32132 entries, 0 to 32132
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher     24081 non-null  object
 1   genres        28850 non-null  object
 2   item_name     32132 non-null  string
 3   release_date  30066 non-null  object
 4   tags          31970 non-null  object
 5   specs         31463 non-null  object
 6   price         30755 non-null  object
 7   early_access  32132 non-null  object
 8   item_id       32132 non-null  int64 
 9   developer     28834 non-null  object
dtypes: int64(1), object(8), string(1)
memory usage: 2.7+ MB


In [19]:
# Notamos que tambien tenemos valores nulos para 'publisher' y 'developer.
# Vamos a reemplazarlos con 'unknown' y cambiaremos el tipo de dato a uno mas conveniente como 'string'
columns = ['publisher', 'developer']
df_games[columns] = df_games[columns].fillna('unknown').astype('string')
df_games.head()

,publisher,genres,item_name,release_date,tags,specs,price,early_access,item_id,developer
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",[Single-player],4.99,False,761140,Kotoshiro
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...","[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...","[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,Poolians.com
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,2017-12-07,"[Action, Adventure, Casual]",[Single-player],0.99,False,767400,彼岸领域
4,unknown,NaN,Log Challenge,NaN,"[Action, Indie, Casual, Sports]","[Single-player, Full controller support, HTC V...",2.99,False,773570,unknown


In [20]:
df_games.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32132 entries, 0 to 32132
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher     32132 non-null  string
 1   genres        28850 non-null  object
 2   item_name     32132 non-null  string
 3   release_date  30066 non-null  object
 4   tags          31970 non-null  object
 5   specs         31463 non-null  object
 6   price         30755 non-null  object
 7   early_access  32132 non-null  object
 8   item_id       32132 non-null  int64 
 9   developer     32132 non-null  string
dtypes: int64(1), object(6), string(3)
memory usage: 2.7+ MB


In [28]:
# Aun nos queda tratar 'release_date'
# Vamos a convertirla a datetime para extaer el año que necesitaremos para nuestra API.
df_games['release_date'] = pd.to_datetime(df_games['release_date'], errors='coerce')

# Extraemos el año
df_games['release_year'] = df_games['release_date'].dt.year

# Para conservar el tipo de dato reemplazamos los nulos con '-1' que no es un año valido en lugar de '0' que si lo es.
df_games['release_year'].fillna(-1, inplace=True)

# Convertimos el tipo de dato a int.
df_games['release_year'] = df_games['release_year'].astype(int)

# Eliminamos 'realease_date' que ya no necesitaremos.
df_games.drop('release_date', axis=1, inplace=True)
df_games.head()

,publisher,genres,item_name,tags,specs,price,early_access,item_id,developer,release_year
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,"[Strategy, Action, Indie, Casual, Simulation]",[Single-player],4.99,False,761140,Kotoshiro,2018
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,"[Free to Play, Strategy, Indie, RPG, Card Game...","[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL,2018
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,"[Free to Play, Simulation, Sports, Casual, Ind...","[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,Poolians.com,2017
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,"[Action, Adventure, Casual]",[Single-player],0.99,False,767400,彼岸领域,2017
4,unknown,NaN,Log Challenge,"[Action, Indie, Casual, Sports]","[Single-player, Full controller support, HTC V...",2.99,False,773570,unknown,-1


In [30]:
df_games.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32132 entries, 0 to 32132
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher     32132 non-null  string
 1   genres        28850 non-null  object
 2   item_name     32132 non-null  string
 3   tags          31970 non-null  object
 4   specs         31463 non-null  object
 5   price         30755 non-null  object
 6   early_access  32132 non-null  object
 7   item_id       32132 non-null  int64 
 8   developer     32132 non-null  string
 9   release_year  32132 non-null  int64 
dtypes: int64(2), object(5), string(3)
memory usage: 2.7+ MB


In [31]:
# Exploramos 'price'
df_games['price']

0                4.99
1        Free To Play
2        Free to Play
3                0.99
4                2.99
             ...     
32128            1.99
32129            4.99
32130            1.99
32131            4.99
32132            4.99
Name: price, Length: 32132, dtype: object

In [32]:
# Notamos que tiene valores no numericos.
# Reemplazamos todos los valores no numéricos en 'price' con 0 y cambiamos el tipo de datos a float
df_games['price'] = pd.to_numeric(df_games['price'], errors='coerce').fillna(0.0).astype(float)

In [33]:
df_games['price']

0        4.99
1        0.00
2        0.00
3        0.99
4        2.99
         ... 
32128    1.99
32129    4.99
32130    1.99
32131    4.99
32132    4.99
Name: price, Length: 32132, dtype: float64

In [34]:
# Damos un vistazo rapido a 'genres y 'tags.
df_games[['genres', 'tags']]

,genres,tags
0,"[Action, Casual, Indie, Simulation, Strategy]","[Strategy, Action, Indie, Casual, Simulation]"
1,"[Free to Play, Indie, RPG, Strategy]","[Free to Play, Strategy, Indie, RPG, Card Game..."
2,"[Casual, Free to Play, Indie, Simulation, Sports]","[Free to Play, Simulation, Sports, Casual, Ind..."
3,"[Action, Adventure, Casual]","[Action, Adventure, Casual]"
4,NaN,"[Action, Indie, Casual, Sports]"
...,...,...
32128,"[Casual, Indie, Simulation, Strategy]","[Strategy, Indie, Casual, Simulation]"
32129,"[Casual, Indie, Strategy]","[Strategy, Indie, Casual]"
32130,"[Indie, Racing, Simulation]","[Indie, Simulation, Racing]"
32131,"[Casual, Indie]","[Indie, Casual, Puzzle, Singleplayer, Atmosphe..."


In [35]:
### Podemos observar que 'tags' contiene tambien a 'genres' por lo que podemos usarla para
# imputar los nulos en 'genres'.
df_games['genres'] = df_games['genres'].fillna(df_games['tags'])

df_games[['genres', 'tags']]

,genres,tags
0,"[Action, Casual, Indie, Simulation, Strategy]","[Strategy, Action, Indie, Casual, Simulation]"
1,"[Free to Play, Indie, RPG, Strategy]","[Free to Play, Strategy, Indie, RPG, Card Game..."
2,"[Casual, Free to Play, Indie, Simulation, Sports]","[Free to Play, Simulation, Sports, Casual, Ind..."
3,"[Action, Adventure, Casual]","[Action, Adventure, Casual]"
4,"[Action, Indie, Casual, Sports]","[Action, Indie, Casual, Sports]"
...,...,...
32128,"[Casual, Indie, Simulation, Strategy]","[Strategy, Indie, Casual, Simulation]"
32129,"[Casual, Indie, Strategy]","[Strategy, Indie, Casual]"
32130,"[Indie, Racing, Simulation]","[Indie, Simulation, Racing]"
32131,"[Casual, Indie]","[Indie, Casual, Puzzle, Singleplayer, Atmosphe..."


In [47]:
# Por ultimo, nos falta reemplazar los nulos restantes en 'genres', 'tags', 'specs' y cambiar su tipo de dato.
columns = ['genres', 'tags', 'specs']
df_games[columns] = df_games[columns].fillna('unknown').astype('string')

# Y convertir 'early_access' a tipo al tipo de datos booleano.
df_games['early_access'] = df_games['early_access'].astype('bool')

In [48]:
df_games.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32132 entries, 0 to 32132
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     32132 non-null  string 
 1   genres        32132 non-null  string 
 2   item_name     32132 non-null  string 
 3   tags          32132 non-null  string 
 4   specs         32132 non-null  string 
 5   price         32132 non-null  float64
 6   early_access  32132 non-null  bool   
 7   item_id       32132 non-null  int64  
 8   developer     32132 non-null  string 
 9   release_year  32132 non-null  int64  
dtypes: bool(1), float64(1), int64(2), string(6)
memory usage: 2.5 MB


In [49]:
# Con nuestro df ya limpio lo exportamos a CSV.
path = 'data/steam_games.csv'
df_games.to_csv(path, index=False, encoding='utf-8')
print(f'El archivo se guardó correctamente en {path}')

El archivo se guardó correctamente en data/steam_games.csv
